# Estimation of Normal HPDI using Optimizer(Tensorflow Interface)

In [1]:
import tensorflow as tf
import external_optimizer
import math

/Users/hyunwoogu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Idea

Given pdf $f_X$ (with $\mu$, $\sigma$), define lower bound $x_l$, and upper bound $x_u$.

**Constraint** : $\int_{x_l}^{x_u} f_X dx$ = .95 ($\because$ 95% credible interval)

**Goal** : *minimize* $x_u - x_l$

Since $f_X$ is normal, we have cdf $F_X$ as follows

$$
\begin{align}
F_X(x) & = \frac{1}{2} [1 + erf(\frac {x - \mu}{\sigma \sqrt{2}})] \\
& \text{where } erf \text{ : error function} \\
\end{align}
$$

Thus, we have

$$
\begin{align}
\int_{x_l}^{x_u} f_X dx & = \frac {1}{2} [erf(\frac {x_u - \mu}{\sigma \sqrt{2}}) - erf(\frac {x_l - \mu}{\sigma \sqrt{2}})] = .95
\end{align}
$$

Define a constraint function $c$

$$
c(x_l, x_u) = \frac {1}{2} [erf(\frac {x_u - \mu}{\sigma \sqrt{2}}) - erf(\frac {x_l - \mu}{\sigma \sqrt{2}})] - .95
$$

Define **Lagrange function** as follows

$$
g(x_l, x_u, \lambda) = (x_u - x_l) + \lambda c(x_l, x_u)
$$

Using the **method of Lagrange multipliers**, we have

$$
\begin{align}
\bigtriangledown g & = \mathbf{0} \\
\frac{\partial g}{\partial x_u} & = 1 + \lambda \frac{1}{\sigma} \sqrt{\frac{2}{\pi}} e ^ {-\frac{(x_u - \mu)^2}{2 \sigma^2}} = 0 \\
\frac{\partial g}{\partial x_l} & = -1 - \lambda \frac{1}{\sigma} \sqrt{\frac{2}{\pi}} e ^ {-\frac{(x_l - \mu)^2}{2 \sigma^2}} = 0 \\
\frac{\partial g}{\partial \lambda} & = \frac {1}{2} [erf(\frac {x_u - \mu}{\sigma \sqrt{2}}) - erf(\frac {x_l - \mu}{\sigma \sqrt{2}})] - .95 = 0 \\
\end{align}
$$

Thus, if we define $v = [x_l, x_u]$,

**Constraint** : $e ^ {-\frac{(x_u - \mu)^2}{2 \sigma^2}} = e ^ {-\frac{(x_l - \mu)^2}{2 \sigma^2}}$

**Goal** : *minimize* $v[1] - v[0]$

### Define Normal PDF

$$
\begin{align}
X & \sim N(10, 3^2) \\[10pt]
f_X & = \frac{1}{3 \sqrt{2 \pi} } e ^ {- \frac{1}{2} (\frac {x-10}{3} )^2}
\end{align}
$$

In [3]:
mean  = 10.
sigma = 3.

### Optimization

In [10]:
initial_guess = tf.Variable([0., 20.], 'vector')

loss      =  tf.subtract(initial_guess[1], initial_guess[0])

cstraint  =[(tf.erf((initial_guess[1] - mean)/(sigma*math.sqrt(2)))-
             tf.erf((initial_guess[0] - mean)/(sigma*math.sqrt(2))))*.5 - .95]

optimizer =  tf.contrib.opt.ScipyOptimizerInterface(loss, equalities=cstraint, method='SLSQP')

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    optimizer.minimize(sess)
    print(sess.run(initial_guess))
    sess.close()

INFO:tensorflow:Optimization terminated with:
  Message: Optimization terminated successfully.
  Objective function value: 11.759783
  Number of iterations: 7
  Number of functions evaluations: 9
[  4.12010813  15.8798914 ]


### Comprison with R code

In [5]:
f = function(x)
{
  dnorm(x, mean=10, sd=3)
}

source('HPDI.R')

HPDI(f)

Interval : [ 4.1415 15.8585 ]
